1. Скачайте датасет MNIST.
2. Изучите выборку (какие там картинки, сколько картинок в каждом классе).
3. Реализуйте загрузку датасета в память с диска с помощью OpenCV. Использовать готовые реализации датасетов и даталоудеров торча запрещено. Пока можно не заморачиваться многопоточностью.
4. Реализуйте аугментацию данных: поворот картинки на определенное кол-во градусов (задается аргументом), зашумление картинки. Реализовывать на numpy. Использовать готовые решения тоже запрещено.
5. Реализуйте способ скормить это нейросети (нормализация, приведение к одному разрешению).
6. Напишите код нейросети. Использовать Sequential запрещено. Модель пишете руками, наследуясь от torch.nn.Module.
7. Реализуйте цикл обучения нейросети. Использовать готовые трейнеры и model.fit() запрещено. Кол-во эпох, learning rate и прочие параметры задаются аргументами.
8. Напишите код для инференса и оценки качества работы нейронной сети.
9. Оценка качества должна заключаться в подсчете Accuracy, Recall, Precision, F-мера. Пользоваться готовыми решениями тоже запрещено.
10. Оформите это как удобную библиотечку, которую можно склонить с гитхаба, положить куда надо файлы весов модели, выборку и обучить / инференсить модель.

In [46]:
import os
import urllib.request
import gzip
import cv2
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optimize
import matplotlib.pyplot as plt

In [50]:
path = "./MNIST/archive/"
os.makedirs(path, exist_ok = True)
#learning_rate = 0.005
epochs = 10
batch_size = 256
seed = 123456
gen = np.random.default_rng(seed)

source = {"training_images" : "http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz",
          "training_labels" : "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz",
          "test_images" : "http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz",
          "test_labels" : "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"
          }

In [51]:
def load():
  data={}

  for key in source:
    data[key] = source[key].split('/')[-1]
    if os.path.exists(path + data[key]):
      print(data[key] + " was here long before you were born...")
    else:
      urllib.request.urlretrieve(source[key], path + data[key])
      print(data[key] + " downloaded right now!")

  images={}
  labels={}

  # Images + Normalization
  for key in ("training_images", "test_images"):
    with gzip.open(os.path.join(path, data[key]), "rb") as mnist:
      images[key] = np.frombuffer(mnist.read(), np.uint8, offset = 16).reshape(-1, 28 * 28) / 255
      print("The shape of {}: {}".format(key, images.shape))

  # Labels
  for key in ("training_labels", "test_labels"):
    with gzip.open(os.path.join(path, data[key]), "rb") as mnist:
      labels[key] = np.frombuffer(mnist.read(), np.uint8, offset = 8)
      print("The shape of {}: {}".format(key, labels.shape))

  #Permutation
  permutation = np.random.permutation(len(labels[key]))

  print(images["training_images"][0])
  print(labels["training_labels"][0])

  images["training_images"] = images[permutation]
  labels["training_labels"] = labels[permutation]
  print(permutation)

  print(images["training_images"][0])
  print(labels["training_labels"][0])

  return images, labels

def noise(image):
  noise = np.random.normal(0.48, 0.05, image.shape)
  gaussedImage = np.clip(image + noise, 0, 1)
  return gaussedImage

def rotate(image):
  angle = np.random.randint(-30, 30)
  theta = np.radians(angle)
  cos, sin = np.cos(theta), np.sin(theta)
  RotMatrix = np.array(((cos, -sin), (sin, cos)))
  image = image.dot(RotMatrix)
  print(RotMatrix)
  image = cv2.resize(image, (28,28))
  return image

def batch(self, images, labels, batch_size):
  samples = images.shape[0]
  batches = samples // batch_size
  for i in range(batches): #np.zeros() if...
    batch_images = np.array([[self.noise(self.rotate(image.reshape(28, 28)))] for image in images[i * batch_size:(i + 1) * batch_size]])
    batch_labels = np.array(labels[i * batch_size:(i + 1) * batch_size])
    return torch.tensor(batch_images), torch.tensor(batch_labels)

In [52]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv = nn.Conv2d(1, 16, kernel_size = (3, 3), stride = 1, padding = 1) # 1 сверточный слой с pooling 4x4 => 7x7 --> ????
    self.maxpool = nn.MaxPool2d(kernel_size = (2, 2), stride = 2) #28x28 --> 3x3 + ReLu() + stride(сдвиг kernel) = 2 + padding = 1 --> 2x2 Pooling
    self.conv2 = nn.Conv2d(16, 32, kernel_size = (3, 3), stride = 1, padding = 1)
    self.maxpool = nn.MaxPool2d(kernel_size = (2, 2), stride = 2) #14x14 --> 3x3 + ReLU() --> 2x2 Pooling
    self.linear = nn.Linear(32 * 7 * 7, 256)
    self.linear2 = nn.Linear(256, 10)

  def forward(self, x_train):
    output = nn.conv(x_train)
    output = self.ReLu(output)
    output = self.maxpool(output)

    output = nn.conv2(output)
    output = self.ReLu(output)
    output = self.maxpool(output)

    output = output.reshape(output.size(0), -1)

    output = self.linear(output)
    output = self.ReLu(output)

    output = self.linear2(output)
    output = self.Softmax(output)

    return output

  def ReLu(x):
    return (x >= 0) * x

  def dReLu(x):
    return x >= 0

  def Softmax(x):
    return np.exp(x)/sum(np.exp(x))

  def dSoftmax(x):
    return np.exp(x)/sum(np.exp(x)) * (1 - np.exp(x) / sum (np.exp(x)))

#Someday...
  #def backward(self, x_train, y_train):
    #error = 2 * (output - y_train) / 
    #return output

  #def Maxpool(x):
    #return

  #def dMaxpool(x):
    #return

In [53]:
def train(cnn, epochs, optimizer, criterion, train_load):
  for epoch in range(epochs):
    training_loss = 0.0
    loss = 0.0
    for i, (images, labels) in enumerate(train_load): #
      output = cnn(images)
      training_loss = criterion(output, labels)

      optimizer.zero_grad()
      training_loss.backward()
      optimizer.step()
      loss += training_loss
      if i % 1000 == 0:
        print(training_loss)
        #print()
  
    print("Epoch ", epoch, " loss is: ", loss)
  print("What must be done is done!")

In [54]:
def test(cnn, test_load): #
  cnn.eval()
  correct_prediction = 0
  true_positive = np.zeros(10, torch.int32)
  false_positive = np.zeros(10, torch.int32)
  false_negative = np.zeros(10, torch.int32)
  with torch.no_grad():
    for images, labels in test_load: #
      output = cnn(input)
      correct_prediction += (torch.max(output.data, 1) == labels).item()

  accuracy = correct_prediction / labels.size(0)
  precision = true_positive / (true_positive + false_positive)
  recall = true_positive / (true_positive + false_negative)
  FScore = 2 * ((precision * recall) / (precision + recall))
  print(( f"Accuracy: {accuracy:.4f}\n"
          f"  Recall: {recall}\n"
          f"  Precision: {precision}"
          f"  F-Score: {FScore:.4f}"
          ))

In [56]:
if __name__ == "__main__":
  #loader = Loader(path) #если в класс обернуть loader
  #TODO: load() func update
  #for key in ("training_labels", "test_labels"):
    #train_load = (Xtrain, Ytrain) = loader.load()
  #for key in ("training_labels", "test_labels"):
    #test_load = (Xtest, Ytest) = loader.load()
  cnn = CNN()
  optimizer = optimize.Adam(cnn.parameters(), lr = 0.005)
  criterion = nn.CrossEntropyLoss()
  #train(cnn, epochs, optimizer, criterion, train_load = (x_train, y_train))
  #test(cnn, test_load = (x_test, y_test))